In [1]:
cog_key = '066c6d22483f4da2976dc29cf2c7dc07'
cog_endpoint = 'https://kaiwenpratice4.cognitiveservices.azure.com/'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Ready to use cognitive services at https://kaiwenpratice4.cognitiveservices.azure.com/ using key 066c6d22483f4da2976dc29cf2c7dc07


In [8]:
from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from python_code import faces
import os
import requests
from bs4 import BeautifulSoup
import bs4 

%matplotlib inline

# Create a face detection client.
face_client = FaceClient(cog_endpoint, CognitiveServicesCredentials(cog_key))
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

Library = {'black_boxer':"黑人拳击手_笑脸", 'yaoming':"姚明_笑脸", 'zhangxueyou_zhangzui':"张学友_吔屎啦你", "jinguanzhang":"金馆长_笑脸"}
Memes = ['black_boxer','yaoming','zhangxueyou_zhangzui','jinguanzhang']
train_path = "C:/Users/5125/Desktop/项目/"

In [9]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

Group = []

for meme in Memes:

    group_id = meme
    try:
        # Delete group if it already exists
        face_client.person_group.delete(group_id)
    except Exception as ex:
        print(ex.message)
    finally:
        face_client.person_group.create(person_group_id=group_id,name=meme, recognition_model='recognition_04')
        print (face_client.person_group.get(group_id))

    # Add a person (Wendell) to the group
    new_meme = face_client.person_group_person.create(group_id, name=meme,recognition_model='recognition_04')
    print(face_client.person_group_person.get(group_id,new_meme.person_id))

    # Get photo's of Wendell
    #folder = os.path.join('data', 'face', 'wendell')
    folder =  "/表情包/" + Library[meme]
    face_1_pics = os.listdir(folder)

    # Register the photos
    i = 0
    fig = plt.figure(figsize=(8, 8))
    for pic in face_1_pics:
        # Add each photo to person in person group
        img_path = os.path.join(folder, pic)
        img_stream = open(img_path, "rb")
        try:
            face_client.person_group_person.add_face_from_stream(group_id, new_meme.person_id, img_stream)
        except:
            pass

    Group.append(new_meme)

    face_client.person_group.train(group_id)
    print(meme + ' Trained!')





{'additional_properties': {}, 'name': 'black_boxer', 'user_data': None, 'recognition_model': None, 'person_group_id': 'black_boxer'}
{'additional_properties': {}, 'name': 'black_boxer', 'user_data': None, 'person_id': '0cf45383-5e29-4f17-9fe9-383967c8e086', 'persisted_face_ids': []}
black_boxer Trained!
{'additional_properties': {}, 'name': 'yaoming', 'user_data': None, 'recognition_model': None, 'person_group_id': 'yaoming'}
{'additional_properties': {}, 'name': 'yaoming', 'user_data': None, 'person_id': '97bf72f4-36fd-4b9e-b67d-1ecce366e4c0', 'persisted_face_ids': []}
yaoming Trained!
{'additional_properties': {}, 'name': 'zhangxueyou_zhangzui', 'user_data': None, 'recognition_model': None, 'person_group_id': 'zhangxueyou_zhangzui'}
{'additional_properties': {}, 'name': 'zhangxueyou_zhangzui', 'user_data': None, 'person_id': 'b87807b0-0387-47b5-818a-718db21bfd21', 'persisted_face_ids': []}
zhangxueyou_zhangzui Trained!
{'additional_properties': {}, 'name': 'jinguanzhang', 'user_data'

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>

In [6]:
image_path = input("输入图片路径:")
#image_path = "C:/Users/5125/Desktop/项目/表情包/黑人拳击手/1.jpg"
image_stream = open(image_path, "rb")

image_faces = face_client.face.detect_with_stream(image=image_stream,recognition_model='recognition_04')
#print(black_boxer.person_id)
#print(image_faces[0].face_id)

image_stream = open(image_path, "rb")
image_face_ids = list(map(lambda face: face.face_id, image_faces))
# Use the Computer Vision service to find text in the image
read_results = computervision_client.recognize_printed_text_in_stream(image_stream)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text 
        print(line_text)

image_text = ""

for meme in Memes:
    #print(meme)
    recognized_faces = face_client.face.identify(image_face_ids, meme)
    for i in recognized_faces:
        if i.candidates != []:
            image_text = meme
    #verify_result_same = face_client.face.verify_face_to_face(image_faces[0].face_id, recognized_faces[0].face_id)
    #print(verify_result_same)

#verify_result_same = face_client.face.verify_face_to_person(image_faces[0].face_id,black_boxer.person_id)
#print(verify_result_same)

105度的你


In [7]:
try:
        html = requests.get("https://jikipedia.com/search?phrase=" + line_text)
except:
        print("Failed")
try:
        soup = BeautifulSoup(html.text, 'html.parser')
        print("表情包人脸：" + Library[image_text])
        print("文字含义: " + soup.find("div",{"class": "brax-render render"}).get_text())
except:
        print("Error")

表情包人脸：张学友_吔屎啦你
文字含义: 指歌曲热爱105℃的你，由阿肆演唱，是一首广告曲。‌​‌​‌​‌‌​‌​‌​‌‌‌‌被一些粉丝拿去做了张哲瀚个人剪辑视频的bgm，在帅哥老婆➕歌曲本身朗朗上口的双重buff下，视频和歌在粉丝群体中迅速传播，因为旋律轻松活泼也被打趣称为菡菡幼儿园园歌。粉丝也屡次提出想听张哲瀚本人演唱该歌曲的愿望，在今日张哲瀚生日直播中实现啦！相似梗:苏破爱豆的笑容都没你的甜～（是该歌第一句歌词）  
